# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 6 2022 10:49AM,245718,10,0085960119,ISDIN Corporation,Released
1,Sep 6 2022 10:49AM,245718,10,0085960121,ISDIN Corporation,Released
2,Sep 6 2022 10:49AM,245718,10,0085960120,ISDIN Corporation,Released
3,Sep 6 2022 10:49AM,245718,10,0085960115,ISDIN Corporation,Released
4,Sep 6 2022 10:49AM,245718,10,0085960114,ISDIN Corporation,Released
5,Sep 6 2022 10:49AM,245718,10,0085960122,ISDIN Corporation,Released
6,Sep 6 2022 10:49AM,245718,10,0085960123,ISDIN Corporation,Released
7,Sep 6 2022 10:49AM,245718,10,0085960126,ISDIN Corporation,Released
8,Sep 6 2022 10:49AM,245718,10,0085960127,ISDIN Corporation,Released
9,Sep 6 2022 10:49AM,245718,10,0085960125,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
18,245712,Released,1
19,245713,Released,42
20,245715,Executing,1
21,245716,Released,62
22,245718,Released,51


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
245712,NaN,1.0
245713,NaN,42.0
245715,1.0,NaN
245716,NaN,62.0
245718,NaN,51.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
245682,2.0,2.0
245689,0.0,7.0
245693,41.0,23.0
245695,0.0,60.0
245696,0.0,49.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
245682,2,2
245689,0,7
245693,41,23
245695,0,60
245696,0,49


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,245682,2,2
1,245689,0,7
2,245693,41,23
3,245695,0,60
4,245696,0,49


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,245682,2,2
1,245689,,7
2,245693,41,23
3,245695,,60
4,245696,,49


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 6 2022 10:49AM,245718,10,ISDIN Corporation
51,Sep 6 2022 10:49AM,245716,10,ISDIN Corporation
113,Sep 6 2022 10:21AM,245715,19,"AdvaGen Pharma, Ltd"
114,Sep 6 2022 10:12AM,245713,15,"Brookfield Pharmaceuticals, LLC"
156,Sep 6 2022 10:09AM,245711,10,ISDIN Corporation
223,Sep 6 2022 10:07AM,245710,10,ISDIN Corporation
278,Sep 6 2022 10:06AM,245709,10,ISDIN Corporation
338,Sep 6 2022 9:59AM,245712,16,"Emersa Waterbox, LLC"
339,Sep 6 2022 9:56AM,245703,10,ISDIN Corporation
400,Sep 6 2022 9:56AM,245707,20,"Exact-Rx, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Sep 6 2022 10:49AM,245718,10,ISDIN Corporation,,51
1,Sep 6 2022 10:49AM,245716,10,ISDIN Corporation,,62
2,Sep 6 2022 10:21AM,245715,19,"AdvaGen Pharma, Ltd",1,
3,Sep 6 2022 10:12AM,245713,15,"Brookfield Pharmaceuticals, LLC",,42
4,Sep 6 2022 10:09AM,245711,10,ISDIN Corporation,,67
5,Sep 6 2022 10:07AM,245710,10,ISDIN Corporation,,55
6,Sep 6 2022 10:06AM,245709,10,ISDIN Corporation,,60
7,Sep 6 2022 9:59AM,245712,16,"Emersa Waterbox, LLC",,1
8,Sep 6 2022 9:56AM,245703,10,ISDIN Corporation,,61
9,Sep 6 2022 9:56AM,245707,20,"Exact-Rx, Inc.",,14


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 6 2022 10:49AM,245718,10,ISDIN Corporation,51,
1,Sep 6 2022 10:49AM,245716,10,ISDIN Corporation,62,
2,Sep 6 2022 10:21AM,245715,19,"AdvaGen Pharma, Ltd",,1
3,Sep 6 2022 10:12AM,245713,15,"Brookfield Pharmaceuticals, LLC",42,
4,Sep 6 2022 10:09AM,245711,10,ISDIN Corporation,67,
5,Sep 6 2022 10:07AM,245710,10,ISDIN Corporation,55,
6,Sep 6 2022 10:06AM,245709,10,ISDIN Corporation,60,
7,Sep 6 2022 9:59AM,245712,16,"Emersa Waterbox, LLC",1,
8,Sep 6 2022 9:56AM,245703,10,ISDIN Corporation,61,
9,Sep 6 2022 9:56AM,245707,20,"Exact-Rx, Inc.",14,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 6 2022 10:49AM,245718,10,ISDIN Corporation,51,
1,Sep 6 2022 10:49AM,245716,10,ISDIN Corporation,62,
2,Sep 6 2022 10:21AM,245715,19,"AdvaGen Pharma, Ltd",,1
3,Sep 6 2022 10:12AM,245713,15,"Brookfield Pharmaceuticals, LLC",42,
4,Sep 6 2022 10:09AM,245711,10,ISDIN Corporation,67,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Sep 6 2022 10:49AM,245718,10,ISDIN Corporation,51.0,NaN
1,Sep 6 2022 10:49AM,245716,10,ISDIN Corporation,62.0,NaN
2,Sep 6 2022 10:21AM,245715,19,"AdvaGen Pharma, Ltd",NaN,1.0
3,Sep 6 2022 10:12AM,245713,15,"Brookfield Pharmaceuticals, LLC",42.0,NaN
4,Sep 6 2022 10:09AM,245711,10,ISDIN Corporation,67.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 6 2022 10:49AM,245718,10,ISDIN Corporation,51.0,0.0
1,Sep 6 2022 10:49AM,245716,10,ISDIN Corporation,62.0,0.0
2,Sep 6 2022 10:21AM,245715,19,"AdvaGen Pharma, Ltd",0.0,1.0
3,Sep 6 2022 10:12AM,245713,15,"Brookfield Pharmaceuticals, LLC",42.0,0.0
4,Sep 6 2022 10:09AM,245711,10,ISDIN Corporation,67.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2948455,491.0,42.0
12,245704,0.0,1.0
15,245713,42.0,0.0
16,245712,1.0,0.0
19,245715,0.0,1.0
20,982784,27.0,2.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2948455,491.0,42.0
1,12,245704,0.0,1.0
2,15,245713,42.0,0.0
3,16,245712,1.0,0.0
4,19,245715,0.0,1.0
5,20,982784,27.0,2.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,491.0,42.0
1,12,0.0,1.0
2,15,42.0,0.0
3,16,1.0,0.0
4,19,0.0,1.0
5,20,27.0,2.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,491.0
1,12,Released,0.0
2,15,Released,42.0
3,16,Released,1.0
4,19,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20
Status,,,,,,
Executing,42.0,1.0,0.0,0.0,1.0,2.0
Released,491.0,0.0,42.0,1.0,0.0,27.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20
0,Executing,42.0,1.0,0.0,0.0,1.0,2.0
1,Released,491.0,0.0,42.0,1.0,0.0,27.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20
0,Executing,42.0,1.0,0.0,0.0,1.0,2.0
1,Released,491.0,0.0,42.0,1.0,0.0,27.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()